In [16]:
import json
import csv

In [19]:
# for i in range(5)
with open('MyData/endsong_0.json') as f:
    df = json.load(f)

UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 37178: character maps to <undefined>